# Fireveg DB imports -- Updated taxonomic list from BIONET

Author: [José R. Ferrer-Paris](https://github.com/jrfep)

Date: 20 August 2024

This Jupyter Notebook includes [Python](https://www.python.org) code to check the status of the Fireveg database.

We will compare two versions with different connection parameters:
- **version 1.0** 
- **version 1.1**


**Please note:**
<div class="alert alert-warning">
    This repository contains code that is intended for internal project management and is documented for the sake of reproducibility.<br/>
    🛂 Only users contributing directly to the project have access to the credentials for data download/upload. 
</div>

## Set-up
### Load modules

In [1]:
# work with paths in operating system
from pathlib import Path
import os, sys
import pandas as pd
from IPython.display import display, Markdown
# Pyprojroot for easier handling of working directory
import pyprojroot

### Define paths for input and output

Define project directory using the `pyprojroot` functions, and add this to the execution path.

In [2]:
repodir = pyprojroot.find_root(pyprojroot.has_dir(".git"))
sys.path.append(str(repodir))

### Load own functions
Load functions from `lib` folder, we will use a function to read db credentials, one for executing database queries and three functions for extracting data from the reference description string

In [3]:
from lib.parseparams import read_dbparams
from lib.firevegdb import dbquery

### Database credentials

🤫 We use a folder named "secrets" to keep the credentials for connection to different services (database credentials, API keys, etc). This checked this folder in our `.gitignore` so that its content are not tracked by git and not exposed. Future users need to copy the contents of this folder manually.

We read database credentials stored in a `database.ini` file using our own `read_dbparams` function.

Compare versions v1.0 and v1.1.

In [4]:
db_v1_0 = read_dbparams(repodir / 'secrets' / 'database.ini', 
                         section='fireveg-db-v1.0')
db_v1_1 = read_dbparams(repodir / 'secrets' / 'database.ini', 
                         section='fireveg-db-v1.1')

## What is in the database

### Field work
Here we query all visits and samples:

In [5]:
qry = """
SELECT survey_name,visit_id,visit_date,CONCAT(givennames,' ', surname) as observer,
COUNT(distinct sample_nr) as samples,
COUNT(distinct record_id) as records
FROM form.field_visit
LEFT JOIN form.observerid
 ON mainobserver=userkey
LEFT JOIN form.field_samples
 USING(visit_id,visit_date)
LEFT JOIN form.quadrat_samples
 USING(visit_id,visit_date,sample_nr)
GROUP BY survey_name,visit_id,visit_date,observer
;
"""

colnames=['survey','site_label','visit_date','observer','samples','records']


In [6]:
res = dbquery(qry,db_v1_0)
site_visits_v1_0 = pd.DataFrame(res,columns=colnames)
res = dbquery(qry,db_v1_1)
site_visits_v1_1 = pd.DataFrame(res,columns=colnames)


We now print a summary of all visits with vegetation records:

In [7]:
ss10=site_visits_v1_0[site_visits_v1_0.records>0]
ss11=site_visits_v1_1[site_visits_v1_1.records>0]

In [18]:
msg="""
**Version {}**: There are {} surveys, {} sites, and {} visits by {} main observers.
There are {} samples and {} records.
"""

In [19]:
prg1 = msg.format(
    "v1.0",
    ss10.survey.unique().size,
    ss10.site_label.unique().size,
    ss10.shape[0],
    ss10.observer.unique().size,
    ss10.samples.sum(),
    ss10.records.sum()
)
prg2 = msg.format(
    "v1.1",
    ss11.survey.unique().size,
    ss11.site_label.unique().size,
    ss11.shape[0],
    ss11.observer.unique().size,
    ss11.samples.sum(),
    ss11.records.sum()
)
display(Markdown(prg1))
display(Markdown(prg2))


**Version v1.0**: There are 10 surveys, 153 sites, and 178 visits by 11 main observers.
There are 1270 samples and 19242 records.



**Version v1.1**: There are 10 surveys, 146 sites, and 170 visits by 10 main observers.
There are 1224 samples and 18397 records.


#### Mallee Woodlands


We compare the results for one of the surveys:


In [24]:
ss10=site_visits_v1_0[site_visits_v1_0['survey'] == 'Mallee Woodlands']
ss11=site_visits_v1_1[site_visits_v1_1['survey'] == 'Mallee Woodlands']

In [25]:
prg1 = msg.format(
    "v1.0",
    ss10.survey.unique().size,
    ss10.site_label.unique().size,
    ss10.shape[0],
    ss10.observer.unique().size,
    ss10.samples.sum(),
    ss10.records.sum()
)
prg2 = msg.format(
    "v1.1",
    ss11.survey.unique().size,
    ss11.site_label.unique().size,
    ss11.shape[0],
    ss11.observer.unique().size,
    ss11.samples.sum(),
    ss11.records.sum()
)
display(Markdown(prg1))
display(Markdown(prg2))


**Version v1.0**: There are 1 surveys, 66 sites, and 93 visits by 7 main observers.
There are 526 samples and 8588 records.



**Version v1.1**: There are 1 surveys, 66 sites, and 93 visits by 7 main observers.
There are 526 samples and 8588 records.


### Species in field samples

In [26]:
qry = """
SELECT DISTINCT species,species_code
FROM form.quadrat_samples
WHERE species_code is not NULL
;
"""
colnames=['species','species_code']

In [27]:
res = dbquery(qry,db_v1_0)
quadrats_v1_0 = pd.DataFrame(res,columns=colnames)
res = dbquery(qry,db_v1_1)
quadrats_v1_1 = pd.DataFrame(res,columns=colnames)

In [28]:
quadrats_v1_1

,species,species_code
0,Pyrrosia rupestris,8163
1,Leucanthemum vulgare,1560
2,Epacris breviflora,2591
3,Goodenia heterophylla,3190
4,Baumea rubiginosa,2302
...,...,...
962,Olearia pimeleoides var. pimeleoides,7258
963,Lophostemon confertus,4242
964,Stephania japonica,3690
965,Viola hederacea,6272


In [29]:
msg="""
**Version {}**: There are {} species with {} unique codes.
"""

In [30]:
prg1 = msg.format(
    'v1.0',
    quadrats_v1_0.species.unique().size,
    quadrats_v1_0.species_code.unique().size
)
prg2 = msg.format(
    'v1.1',
    quadrats_v1_1.species.unique().size,
    quadrats_v1_1.species_code.unique().size
)
display(Markdown(prg1))
display(Markdown(prg2))


**Version v1.0**: There are 1043 species with 1047 unique codes.



**Version v1.1**: There are 966 species with 966 unique codes.


### Traits from the literature

Read trait info into a data frame.

In [57]:
qry = "select code,name,description,value_type,life_stage,life_history_process,category_vocabulary,method_vocabulary from litrev.trait_info ;"
res = dbquery(qry,db_v1_1)
data = pd.DataFrame(res)
trait_info=data.rename(columns={0:"Trait code", 1:"Trait name", 2:"Description", 4:"Life stage", 5:"Life history process",  3:"Value type",6:"category_vocabulary",7:"method_vocabulary"})


In [58]:
trait_info.shape

(38, 8)

Count how many are categorical, etc.

In [60]:
trait_info['Value type'].value_counts()

Value type
categorical    14
numerical      10
TBA             6
numeric         4
TO DO           3
text            1
Name: count, dtype: int64

In [61]:
trait_info['Life stage'].value_counts()

Life stage
Standing plant    21
Seed              13
Seedling           4
Name: count, dtype: int64

In [62]:
trait_info['Life history process'].value_counts()

Life history process
Reproduction    11
Germination     10
Survival         7
Growth           5
Recruitment      4
Dispersal        1
Name: count, dtype: int64

Overview of sources of trait data

In [36]:
qry="""
    SELECT 'repr2' AS table_name, main_source, species, species_code FROM litrev.repr2
    UNION SELECT 'germ8' AS table_name, main_source, species, species_code FROM litrev.germ8
    UNION SELECT 'rect2' AS table_name, main_source, species, species_code FROM litrev.rect2
    UNION SELECT 'germ1' AS table_name, main_source, species, species_code FROM litrev.germ1
    UNION SELECT 'grow1' AS table_name, main_source, species, species_code FROM litrev.grow1
    UNION SELECT 'repr4' AS table_name, main_source, species, species_code FROM litrev.repr4
    UNION SELECT 'surv5' AS table_name, main_source, species, species_code FROM litrev.surv5
    UNION SELECT 'surv6' AS table_name, main_source, species, species_code FROM litrev.surv6
    UNION SELECT 'surv7' AS table_name, main_source, species, species_code FROM litrev.surv7
    UNION SELECT 'disp1' AS table_name, main_source, species, species_code FROM litrev.disp1
    UNION SELECT 'repr3' AS table_name, main_source, species, species_code FROM litrev.repr3a
    UNION SELECT 'repr3a' AS table_name, main_source, species, species_code FROM litrev.repr3
    UNION SELECT 'surv4' AS table_name, main_source, species, species_code FROM litrev.surv4
    UNION SELECT 'surv1' AS table_name, main_source, species, species_code FROM litrev.surv1
;
"""
colnames=['traits', 'source', 'species', 'species_code']


In [37]:
res = dbquery(qry,db_v1_0)
lit_traits_v1_0 = pd.DataFrame(res,columns=colnames)
res = dbquery(qry,db_v1_1)
lit_traits_v1_1 = pd.DataFrame(res,columns=colnames)

In [38]:
msg = """
**Version {}**: There are {} records from {} main sources. 
They include {} traits for {} species/taxa with {} unique species codes.
"""

In [40]:
prg1=msg.format(
    'v1.0',
    lit_traits_v1_0.shape[0],
    lit_traits_v1_0.source.unique().size,
    lit_traits_v1_0.traits.unique().size,
    lit_traits_v1_0.species.unique().size,
    lit_traits_v1_0.species_code.unique().size)
prg2=msg.format(
    'v1.1',
    lit_traits_v1_1.shape[0],
    lit_traits_v1_1.source.unique().size,
    lit_traits_v1_1.traits.unique().size,
    lit_traits_v1_1.species.unique().size,
    lit_traits_v1_1.species_code.unique().size)
display(Markdown(prg1))
display(Markdown(prg2))


**Version v1.0**: There are 30414 records from 5 main sources. 
They include 14 traits for 12591 species/taxa with 7253 unique species codes.



**Version v1.1**: There are 41836 records from 4 main sources. 
They include 14 traits for 18009 species/taxa with 7677 unique species codes.


In [53]:
lit_traits_v1_0.groupby(['source','traits']).agg('count')

species  species_code
source                   traits                       
Bell Vollmer Gellie 1993 surv1         1             1
NSWFFRDv2.1              germ1      1596          1596
                         grow1        32            32
                         rect2      1026          1026
                         repr2       138           138
                         repr3       655           655
                         repr3a      830           830
                         repr4        21            21
                         surv1      3044          3044
                         surv4      1257          1257
                         surv5      1196          1196
                         surv6         7             7
                         surv7        87            87
Ooi Myerscough Auld 2007 germ8       188           164
austraits-3.0.2          disp1      9876          6253
                         surv1     10459          6555

In [52]:
lit_traits_v1_1.groupby(['source','traits']).agg('count')

species  species_code
source                   traits                       
Bell Vollmer Gellie 1993 surv1         1             1
NSWFFRDv2.1              germ1      1596          1596
                         grow1        32            32
                         rect2      1026          1026
                         repr2       138           138
                         repr3       655           655
                         repr3a      830           830
                         repr4        21            21
                         surv1      3044          3044
                         surv4      1257          1257
                         surv5      1196          1196
                         surv6         7             7
                         surv7        87            87
austraits-6.0.0          disp1     11246          6412
                         germ1      3230          1746
                         germ8      4077          1089
                         repr2       399            97
                         surv1     12993          6591

Define queries we are going to use multiple times:

In [119]:
# Check comments on vocabularies
qry_vocabulary = "SELECT pg_catalog.obj_description(t.oid, 'pg_type')::json from pg_type t where typname = '%s';" 

# Number of records per source
qry_source = 'SELECT main_source,count(*) FROM litrev.%s GROUP BY main_source'
# Number of records per value of categorical variable
qry_values = ' select norm_value,count(*),count(distinct species),count(distinct species_code) from litrev.%s group by norm_value;'

# Number of records per value of numerical variable
qry_triplet = ' select best is NOT NULL as b, lower is NOT NULL as l, upper is NOT NULL as u,count(*),count(distinct species),count(distinct species_code) from litrev.%s group by b,l,u;'

# Raw values when norm value is NULL 
qry_nulls = ' select raw_value,count(*),count(distinct species),count(distinct species_code) from litrev.%s where norm_value is NULL group by raw_value;'
# Raw values when best/lower/upper are all NULL 
qry_triplet_nulls = 'select raw_value,count(*),count(distinct species),count(distinct species_code) from litrev.%s where best is NULL and lower is NULL and upper is NULL group by raw_value;'


In [139]:
def show_trait_info(trait_name,trait_df,params):
    elem = trait_df[trait_df["Trait code"]==trait_name]
    msg = "## Trait `{}`\n***{}*** {}.\n\n_Life Stage_: {} / _Life history process_: {}" .format(
        trait_name,
        elem.iloc[0]['Trait name'],
        elem.iloc[0]['Description'] ,
        elem.iloc[0]['Life stage'] ,
        elem.iloc[0]['Life history process'] )                
    display(Markdown(msg))
    # display(elem.transpose())
    cat_vocab = elem.iloc[0]['category_vocabulary']
    if cat_vocab is not None:
        cat_table = dbquery(qry_vocabulary % cat_vocab , params)
        display(Markdown("### Vocabulary for trait"))
        display(pd.DataFrame(cat_table[0]).transpose())
    met_vocab = elem.iloc[0]['method_vocabulary']
    if met_vocab is not None:
        met_table = dbquery(qry_vocabulary % met_vocab,params)
        display(Markdown("### Vocabulary for the methods"))
        display(pd.DataFrame(met_table[0]).transpose())
    display(Markdown("### Summary of data"))
    if elem.iloc[0]['Value type'] == 'categorical':
        res = dbquery(qry_values % trait_name,params)
        data=pd.DataFrame(res,
                      columns=["Value","Nr. records","Nr. taxa","Nr. valid"])
        display(data)
        display(Markdown("Transcription errors"))
        res=dbquery(qry_nulls % trait_name,params)
        nulldata=pd.DataFrame(res,
                      columns=["Value","Nr. records","Nr. taxa","Nr. valid"])
        display(nulldata)
    if elem.iloc[0]['Value type'] == 'numerical':
        res = dbquery(qry_triplet % trait_name,params)
        data=pd.DataFrame(res,
                      columns=["best","lower","upper","Nr. records","Nr. taxa","Nr. valid"])
        display(data)
        display(Markdown("Transcription errors"))
        res=dbquery(qry_triplet_nulls % trait_name,params)
        nulldata=pd.DataFrame(res,
                      columns=["Value","Nr. records","Nr. taxa","Nr. valid"])
        display(nulldata)


In [140]:
for tt in ['surv1','surv4','surv5','surv6','surv7',
           'disp1',
           'rect2',
           'germ1','germ8',
           'grow1',
           'repr2','repr3','repr3a','repr4']:
    show_trait_info(tt,trait_info,db_v1_1)

## Trait `surv1`
***Resprouting - full canopy scorch*** Ordinal categories of survival and resprouting proportions for plants subjected to 100% canopy scorch.

_Life Stage_: Standing plant / _Life history process_: Survival

### Vocabulary for trait

,0
None,< 5 % individuals in a population resprout aft...
Few,> 5 and < 30 % individuals in a population res...
Half,> 30 and < 70 % individuals in a population re...
Most,> 70 and < 90 % individuals in a population re...
All,> 90 % individuals in a population resprout af...
Unknown,No data


### Vocabulary for the methods

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
Estimated by extrapolation from observed values,Estimated by extrapolation from (time series) ...
Inferred from plant / organ / growth stage morphology,Inference based on relevant morphological stru...
Inferred from related taxa,Assigned to same category as congeners or conf...
Manipulative experiment,Manipulative experiment. Individuals sampled b...
Qualitative estimate,Guestimate from visual assessment of post-fire...
Unspecified methods,Methods unclear or undocumented.


...

,0
Direct field observation or measure (unknown sample size),Estimates based on data observed or measured i...
Direct observation (small sample),Estimates based on data observed or measured i...
Direct observation (large sample),Estimates based on data observed or measured i...
Estimated by extrapolation from observed values,Estimated by extrapolation from (time series) ...
"Trait evidence, but start date uncertain","Evidence of trait expression, but age of first..."
Inferred form other trait,"Inferred from values of a different, but relat..."
Inferred from partial data,Inferred from incomplete or indirect informati...
No evidence of trait expression by specified age,No evidence of trait expression by specified a...
Unspecified methods,Methods unclear or undocumented.


### Summary of data

,best,lower,upper,Nr. records,Nr. taxa,Nr. valid
0,False,False,False,1,1,1
1,False,False,True,3,3,3
2,False,True,False,1,1,1
3,False,True,True,7,7,7
4,True,False,False,9,9,9
5,True,False,True,1,1,1
6,True,True,False,1,1,1


Transcription errors

,Value,Nr. records,Nr. taxa,Nr. valid
0,[Prim Juv period - 6 for 50% of population to ...,1,1,1


# References
This represents the existing sources data stream

In [6]:
qrystr = "select count(*) from litrev.ref_list;"
dbquery(qrystr,db_v1_0)

[[309]]

In [7]:
dbquery(qrystr,db_v1_1)

[[347]]